<a href="https://colab.research.google.com/github/sergegutier/machine-learning/blob/master/cnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Convolutional Neural Networks - Image Classifier**

In [0]:
!pip install tensorflow==1.14.0

Preprocessing - We must set the working directory with Google Drive

In [0]:
#Mount Drive to Colab

from google.colab import drive 
drive.mount('/content/gdrive')

In [0]:
import os
os.chdir("/content/gdrive/My Drive/Colab Notebooks/dataset") 
os.getcwd()

## Part I - Building the CNN

In [0]:
from keras.models import Sequential # Makes the ANN to initalize NN
from keras.layers import Convolution2D # Adds convolutional layers - 2D is for images
from keras.layers import MaxPooling2D # Adds pooling layers
from keras.layers import Flatten #Convert pooled maps into flattened feature vector
from keras.layers import Dense # Adds the fully connected layers in ANN

Intialize the CNN

In [0]:
classifier = Sequential() # Since we classify images, we use classifier as a name

Step 1 - Convolution

In [0]:
classifier.add(Convolution2D(32,3,3, input_shape = (64,64,3), activation='relu')) # #32 feature detectors, of 3x3 measures; for input-shape, we used 3 channels (RGB) and 64x64 - parameters are inverted in tensorflow

Step 2 - Pooling

In [0]:
classifier.add(MaxPooling2D(pool_size=(2,2)))

Note : We are adding a *second* convolutional layer, after we decided to improve upon 76% accuracy

In [0]:
classifier.add(Convolution2D(32,3,3, activation='relu')) # we removed input_shape because we are not working with images in this step, but rather already pooled features 
classifier.add(MaxPooling2D(pool_size=(2,2)))

Step 3 - Flattening

In [0]:
classifier.add(Flatten())

Step 4 - Full Connection

In [0]:
classifier.add(Dense(output_dim = 128, activation='relu'))
classifier.add(Dense(output_dim = 1, activation = 'sigmoid'))

Step 5 - Compile CNN

In [0]:
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics=['accuracy'])

## Part II - Fitting CNN to Images

In [0]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

In [0]:
test_datagen = ImageDataGenerator(rescale=1./255)

training_set = train_datagen.flow_from_directory(
        'training_set',
        target_size=(64, 64),
        batch_size=32,
        class_mode='binary')
test_set = test_datagen.flow_from_directory(
        'test_set',
        target_size=(64, 64),
        batch_size=32,
        class_mode='binary')

In [0]:
classifier.fit_generator(
        training_set,
        samples_per_epoch=8000,
        nb_epoch=25,
        validation_data=test_set,
        nb_val_samples=2000)


## Part III - Making a Single Prediction

Step 1 - We load the image and convert it to an array 

In [0]:
import numpy as np
from keras.preprocessing import image

os.chdir("/content/gdrive/My Drive/Colab Notebooks/dataset/single_prediction") # for now, we have to readjust the directory
os.getcwd()

test_image = image.load_img('teeny1.jpg',target_size=(64,64)) #We load the image

test_image = image.img_to_array(test_image)

test_image = np.expand_dims(test_image, axis = 0) # we have to add another dimension so we can predict. The predict method does it by multiple batches

result = classifier.predict(test_image) # we store the binary result in this variable

Step 2 - Determine wether '1' result is for dog or cat

In [0]:
training_set.class_indices # this prints : cats = 0, dogs = 1

if result [0][0] == 1:
  prediction = 'Dog'
else :
  prediction = 'Cat'

  print("This is a "+prediction)